In [78]:
import pandas as pd
import numpy as np
import datetime
import re
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import graphviz
import pydotplus
from IPython.display import Image
from sklearn.externals.six import StringIO

In [4]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [10]:
dummy_cols = df_train.select_dtypes('object').columns
for col in dummy_cols:
    categories = df_train[col].unique()

    df_train[col] = pd.Categorical(df_train[col], categories=categories)
    df_test[col] = pd.Categorical(df_test[col], categories=categories)

df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)

In [67]:
print(x_test.dtypes)
print(pd.DataFrame(y_test_f, index=list(x_test_f.index)))

age                   int64
bmi                 float64
children              int64
sex_male              uint8
sex_female            uint8
smoker_no             uint8
smoker_yes            uint8
region_southeast      uint8
region_southwest      uint8
region_northwest      uint8
region_northeast      uint8
dtype: object
     0
0    0
1    2
2    0
3    0
4    0
..  ..
395  0
396  0
397  2
398  0
399  0

[400 rows x 1 columns]


In [41]:
x.iloc[:,1]

0       32.665465
1       29.798725
2       32.722029
3       38.429831
4       29.641854
          ...    
1595    32.772830
1596    37.189564
1597    43.281979
1598    28.232308
1599    28.997113
Name: bmi, Length: 1600, dtype: float64

In [71]:
x = df_train.drop(["id","charges"],axis=1)
y = df_train["charges"]
x_test_f = df_test.drop("id", axis=1)



x_train, x_test, y_train, y_test = train_test_split(x,y,shuffle=True, random_state=31, stratify=y)
lr_tree = DecisionTreeClassifier(max_depth=4)
lr_tree.fit(x_train,y_train)
print(lr_tree.score(x_test, y_test))

dot_data = export_graphviz(lr_tree)
graph = graphviz.Source(dot_data)
graph.render('test', format='png')

y_test_f = lr_tree.predict(x_test_f)
print(len(y_test_f))

df_submit = pd.DataFrame(y_test_f, index=list(df_test["id"]))
df_submit.to_csv("submit.csv", header = False)
print(df_submit)

0.88
400
      0
13    0
23    2
27    0
28    0
29    0
...  ..
1968  0
1972  0
1979  2
1983  0
1995  0

[400 rows x 1 columns]


In [89]:
class data_analysis:
    def __init__(self, df_train_name, df_test_name, header_ck = True):
        self.df_train = pd.read_csv(df_train_name, header = header_ck)
        self.df_test = pd.read_csv(df_test_name, header = header_ck)
        
    # 使用する変数と目的変数を指定して、抽出および説明変数と目的変数で分離
    def extract_df(self, extract_list, object_value):
        self.df_train_x = self.df_train.drop(extract_list, axis=1)
        self.df_train_y = self.df_train(object_value)
        self.df_test_x = self.df_test.drop(extract_list, axis=1)
        
    # validation
    def validation_product(self, size = 0.75, stratify = True):
        if stratify == True:
            self.x_train, self.x_valid, self.y_train, self.y_valid = train_test_split(self.df_train_x, self.df_train_y, train_size=size, shuffle=True, random_state=31, stratify=self.df_train_y)
        else:
            self.x_train, self.x_valid, self.y_train, self.y_valid = train_test_split(self.df_train_x, self.df_train_y, train_size=size, shuffle=True, random_state=31)

    # XGBoostにおけるグリッドサーチによるパラメータ推定, task=["classify", "regression"]
    def xgb_estimate_params(self, task = "classify", binary_task = True):
        if task == "classify":
            params = {
                "learning_rate":np.arange(0.1,0.6,0.1),
                "max_depth":np.arange(5,8,1),
                "subsample":np.arange(0.6,1.1,0.1),
                "colsample_bytree":np.arange(0.6,1.1,0.1),
                "reg_lambda":np.arange(0.6,1.1,0.1)
            }
            if binary_task:
                clf_xgb = xgb.XGBClassifier(objective="binary:logistic")
            else:
                clf_xgb = xgb.XGBClassifier(objective="multi:softmax")
            
            kF = KFold(n_splits = 5, shuffle=True, random_state=31)
            self.gs = GridSearchCV(clf_xgb, params, cv = kF, n_jobs=-1, scoring = "accuracy")
            self.gs.fit(self.df_train_x,self.df_train_y)
            print("{}, {}".format(self.gs.best_estimator_, self.gs.best_score_))
    
    # XGBoostによるテスト
    def xgb_test(self, signate = False, col_list):
        self.df_test_y = self.gs.predict(self.df_test_x)
        if signate = True:
            submit2signate(self.df_test)
    


/Users/satoshi/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:48:01] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 2 0
 0 2 0 0 2 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1 0 0 1 0 0 0 0 0 1 1 0
 2 0 0 0 0 1 0 0 0 0 0 0 0 0 2 0 0 0 0 1 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 1 0 0 0 0 2 0 0 0 0 0 0 0 2 0 1 1
 0 0 0 0 0 2 0 1 0 0 0 2 0 2 0 1 0 0 1 0 0 0 0 2 2 0 0 0 0 0 0 0 0 0 0 0 2
 0 0 2 0 0 0 0 0 0 0 0 2 2 0 1 0 0 2 0 0 0 2 0 0 0 1 0 0 1 0 0 0 0 0 0 2 0
 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0
 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 2 0 0 0 0 0 1 0 2 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 2 0 0 0 2 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 2 0 0 0 0 0
 0 0 0 0 0 0 0 0 2 0 2 0 0 0 1 0 0 0 0 0 0

In [97]:
print(gs.best_estimator_)
print(gs.best_score_)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.30000000000000004, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=0.7,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)
0.8608333333333332


In [ ]:
def xgb_estimate_params(df_train):
    x = df_train.drop(["id","charges"],axis=1)
    y = df_train["charges"]

    x_train, x_test, y_train, y_test = train_test_split(x,y,shuffle=True, random_state=31, stratify=y)

    params = {
        "learning_rate":[0.3]
        "max_depth":[5],
        "subsample":[0.7]
    #     "learning_rate":np.arange(0.1,0.6,0.1),
    #     "max_depth":np.arange(5,8,1),
    #     "subsample":np.arange(0.6,1.1,0.1),
    #     "colsample_bytree":np.arange(0.6,1.1,0.1),
    #     "reg_lambda":np.arange(0.6,1.1,0.1)
    }

    x_train, x_test, y_train, y_test = train_test_split(x,y,shuffle=True, random_state=31, stratify=y)

    clf_xgb = xgb.XGBClassifier(objective="multi:softmax")
    kF = KFold(n_splits = 5, shuffle=True, random_state=31)
    gs = GridSearchCV(clf_xgb, params, cv = kF, n_jobs=-1, scoring = "accuracy")
    kF = KFold(n_splits = 5, shuffle=True, random_state=31)
    # score = cross_validate(clf_xgb, x, y, cv=kF, scoring="accuracy")
    gs.fit(x_train,y_train)
    print(gs.predict(x_test))

def xgb_test(df_test):
    x_test_f = df_test.drop("id", axis=1)
    params = {
        "learning_rate":[0.3]
        "max_depth":[5],
        "subsample":[0.7]
    }
